### INIT

In [2]:
import sys
from pathlib import Path
from multiversx_sdk import ApiNetworkProvider, ProxyNetworkProvider

sys.path.append(str(Path.cwd().parent.parent.absolute()))
import config
from context import Context
from utils.utils_chain import WrapperAddress


SIMULATOR_URL = "http://localhost:8085"
SIMULATOR_API = "http://localhost:3001"

GENERATE_BLOCKS_URL = f"{SIMULATOR_URL}/simulator/generate-blocks"
GENERATE_BLOCKS_UNTIL_EPOCH_REACHED_URL = f"{SIMULATOR_URL}/simulator/generate-blocks-until-epoch-reached"
PROJECT_ROOT = Path.cwd().parent.parent
proxy = ProxyNetworkProvider(SIMULATOR_URL)
api = ApiNetworkProvider(SIMULATOR_API)
DOCKER_URL = PROJECT_ROOT / "docker"

context = Context()
context.network_provider.proxy = proxy
context.network_provider.api = api

wasm_path = "wasm/fees-collector/fees-collector.wasm"
contract_code_hash = "ae5c0171d34f2bac384b6d38184f89abd7ef05a4d5d9b973db3c58015bdf587f"

2025-05-08 15:31:00,043 - DEBUG - Account.sync_nonce() done: 4286


Shadowfork detected. Owner: erd1ss6u80ruas2phpmr82r42xnkd6rxy40g9jl69frppl4qez9w2jpsqj8x97
loaded 10 accounts from 1 PEM files.


2025-05-08 15:31:00,145 - DEBUG - Account.sync_nonce() done: 4286
2025-05-08 15:31:00,146 - WARNING - Loaded 16 tokens instead of expected 15.
2025-05-08 15:31:00,248 - DEBUG - Account.sync_nonce() done: 4286
2025-05-08 15:31:00,250 - INFO - Loaded 3 egld_wraps out of expected 3.
2025-05-08 15:31:00,251 - INFO - Loaded 1 locked_assets out of expected 1.
2025-05-08 15:31:00,252 - INFO - Loaded 1 proxies out of expected 1.
2025-05-08 15:31:00,252 - INFO - Loaded 0 simple_locks out of expected 0.
2025-05-08 15:31:00,253 - INFO - Loaded 1 simple_locks_energy out of expected 1.
2025-05-08 15:31:00,254 - INFO - Loaded 1 fees_collectors out of expected 1.
2025-05-08 15:31:00,254 - INFO - Loaded 1 unstakers out of expected 1.
2025-05-08 15:31:00,255 - INFO - Loaded 1 proxies_v2 out of expected 1.
2025-05-08 15:31:00,256 - INFO - Loaded 1 position_creator out of expected 1.
2025-05-08 15:31:00,256 - INFO - Loaded 1 locked_token_position_creator out of expected 1.
2025-05-08 15:31:00,257 - INFO 

### CHAIN SIM CONFIG - FEES COLLECTOR

In [6]:
from tools.chain_simulator_connector import ChainSimulator, start_handler
from argparse import Namespace

docker_path = config.HOME / "Projects/testing/full-stack-docker-compose/chain-simulator"
state_path = config.DEFAULT_WORKSPACE / "states"
args = Namespace(docker_path=str(docker_path), state_path=str(state_path))
chain_sim, found_accounts = start_handler(args)

2025-05-08 15:44:10,552 - WARNING - State path is not provided or does not exist. Using default folder: states


FileNotFoundError: [Errno 2] No such file or directory: 'states'

In [2]:
USERS = [
        "erd1emxytu3umnzm4k2cn2xmtppy8j3dm3lnsjhfzkul8gd5a4xxuk3qsl4xjw", 
        "erd1rv5twgkz5uatdvgk5ymzmgzmz38dxqh8agvlkt97mfun8hn4x4xqyptslm",
        "erd1njvcr0r89km6pexrxh0d6h36pkeuwr5j042e7l46l3mdlxvwhejsfz9w4n",
        "erd1ss6u80ruas2phpmr82r42xnkd6rxy40g9jl69frppl4qez9w2jpsqj8x97" # DEX OWNER
]

BASE_TOKEN = "MEX-455c57"

In [4]:
import json
import subprocess
from time import sleep
from typing import Any

from utils.utils_chain import Account


def load_accounts_state(project_root: Path, addresses: list[str]) -> list[dict[str, Any]]:
    states = []
    
    for address in addresses:
        print(f"Loading state for {address}")
        user_path = f"0_{address}_0_chain_config_state.json"
        system_account_path = f"0_system_account_state_{address}.json"
        
        user_file = project_root / "states" / user_path
        system_file = project_root / "states" / system_account_path
        
        if user_file.exists():
            with open(user_file, "r") as file:
                user_state = json.load(file)
                if user_state:
                    print(f"Found {user_file.name}")
                    states.append(user_state)
                
        if system_file.exists():
            with open(system_file, "r") as file:
                system_state = json.load(file)
                if system_state:
                    print(f"Found {system_file.name}")
                    states.append(system_state)
            
    return states
    
def apply_states(proxy: ProxyNetworkProvider, states: list[dict[str, Any]]):
    for state in states:
        proxy.do_post_generic(f"{SIMULATOR_URL}/simulator/set-state", state)

# @pytest.fixture
def load_and_apply_state(proxy: ProxyNetworkProvider, project_root: Path, owner: str, users: list[str]):
    # Load and set state for all keys
    with open(project_root / "states" / "0_all_all_keys.json", "r") as file:
        retrieved_state = json.load(file)
        apply_states(proxy, [retrieved_state])

    # Load owner and users state
    accounts = [owner]
    accounts.extend(users)
    states = load_accounts_state(project_root, accounts)
    apply_states(proxy, states)
        

def advance_blocks(number_of_blocks: int):
    proxy.do_post_generic(f"{GENERATE_BLOCKS_URL}/{number_of_blocks}", {})

def advance_epoch(number_of_epochs: int):
    proxy.do_post_generic(f"{GENERATE_BLOCKS_URL}/{number_of_epochs * 20}", {})

def advance_to_epoch(epoch: int):
    proxy.do_post_generic(f"{GENERATE_BLOCKS_UNTIL_EPOCH_REACHED_URL}/{epoch}", {})

def users_init() -> list[Account]:
    print(context.deployer_account.address.bech32())
    context.deployer_account.sync_nonce(context.network_provider.proxy)

    users = []
    for user in USERS:
        user_account = Account(pem_file=config.DEFAULT_ACCOUNTS)
        user_account.address = WrapperAddress(user)
        user_account.sync_nonce(context.network_provider.proxy)
        users.append(user_account)

    return users

### CONTRACTS

In [5]:
from utils.contract_data_fetchers import FeeCollectorContractDataFetcher
from contracts.router_contract import RouterContract
from contracts.simple_lock_energy_contract import SimpleLockEnergyContract
from contracts.fees_collector_contract import FeesCollectorContract
from contracts.pair_contract import PairContract
from utils.contract_retrievers import retrieve_pair_by_address

fees_collector_contract: FeesCollectorContract
fees_collector_contract = context.get_contracts(config.FEES_COLLECTORS)[0]

config.DEFAULT_PROXY =  "https://gateway.multiversx.com"

energy_contract: SimpleLockEnergyContract
router_contract: RouterContract
pair_contract: PairContract

pair_contract = retrieve_pair_by_address("erd1qqqqqqqqqqqqqpgq0lzzvt2faev4upyf586tg38s84d7zsaj2jpsglugga")    # operating pair
mex_contract = retrieve_pair_by_address("erd1qqqqqqqqqqqqqpgqa0fsfshnff4n76jhcye6k7uvd7qacsq42jpsp6shh2") # egldmex contract
energy_contract = context.get_contracts(config.SIMPLE_LOCKS_ENERGY)[0]
router_contract = context.deploy_structure.get_deployed_contract_by_index(config.ROUTER_V2, 0)

print(fees_collector_contract.address)
print(router_contract.address)
print(pair_contract.address)
print(mex_contract.address)


erd1qqqqqqqqqqqqqpgqjsnxqprks7qxfwkcg2m2v9hxkrchgm9akp2segrswt
erd1qqqqqqqqqqqqqpgqq66xk9gfr4esuhem3jru86wg5hvp33a62jps2fy57p
erd1qqqqqqqqqqqqqpgq0lzzvt2faev4upyf586tg38s84d7zsaj2jpsglugga
erd1qqqqqqqqqqqqqpgqa0fsfshnff4n76jhcye6k7uvd7qacsq42jpsp6shh2


In [6]:
from utils.utils_chain import Account, WrapperAddress as Address

users = users_init()
user_account = Account(users[0].address.to_bech32())
user_account.sync_nonce(context.network_provider.proxy)
user_account.signer = context.deployer_account.signer

2025-04-24 17:12:55,080 - DEBUG - Account.sync_nonce() done: 4308
2025-04-24 17:12:55,085 - DEBUG - Account.sync_nonce() done: 2659
2025-04-24 17:12:55,090 - DEBUG - Account.sync_nonce() done: 986
2025-04-24 17:12:55,098 - DEBUG - Account.sync_nonce() done: 12332
2025-04-24 17:12:55,103 - DEBUG - Account.sync_nonce() done: 4308
2025-04-24 17:12:55,106 - DEBUG - Account.sync_nonce() done: 2659


erd1ss6u80ruas2phpmr82r42xnkd6rxy40g9jl69frppl4qez9w2jpsqj8x97


In [ ]:
import json
with open(PROJECT_ROOT / "states" / "0_erd1ss6u80ruas2phpmr82r42xnkd6rxy40g9jl69frppl4qez9w2jpsqj8x97_0_chain_config_state.json", "r") as file:
    json_account = json.load(file)
proxy.do_post_generic(f"{SIMULATOR_URL}/simulator/set-state", json_account)

In [ ]:
import json
with open(PROJECT_ROOT / "states" / "0_fees_collectors_0_chain_config_state.json", "r") as file:
    json_account = json.load(file)
proxy.do_post_generic(f"{SIMULATOR_URL}/simulator/set-state", json_account)

In [ ]:
advance_epoch(7)

In [ ]:
advance_blocks(5)

CLAIM REWARDS

In [ ]:
user_account.sync_nonce(context.network_provider.proxy)

fees_collector_contract.claim_rewards(user_account, proxy) #4 blocks
advance_blocks(4)

CLAIM BOOSTED

In [ ]:
fees_collector_contract.claim_boosted_rewards(user_account, proxy)

ADD ADMIN

In [ ]:
from utils.utils_chain import WrapperAddress

context.deployer_account.sync_nonce(proxy)
fees_collector_contract.add_admin(context.deployer_account, proxy, [context.deployer_account.address])

REMOVE ADMIN

In [ ]:
from utils.utils_chain import WrapperAddress

context.deployer_account.sync_nonce(proxy)
fees_collector_contract.remove_admin(context.deployer_account, proxy, [context.deployer_account.address])

REDISTRIBUTE REWARDS

In [ ]:

context.deployer_account.sync_nonce(proxy)
fees_collector_contract.redistribute_rewards(context.deployer_account, proxy)
advance_blocks(1)

SET ADMINS

In [ ]:
fees_collector_contract.add_admin(context.deployer_account, proxy, [context.deployer_account.address])

In [10]:
week = fees_collector_contract.get_current_week(proxy)
user_energy = fees_collector_contract.get_user_energy_for_week(user_account.address.to_bech32(), proxy, week)
last_active_week = fees_collector_contract.get_last_active_week_for_user(user_account.address.to_bech32(), proxy)
print(week)
print(user_energy)
print(last_active_week)

130
{'amount': 378166121477748896610405802164, 'last_update_epoch': 1767, 'total_locked_tokens': 410821713011768735240637278}
130


SET BURN PERCENTAGE

In [ ]:
context.deployer_account.sync_nonce(proxy)
fees_collector_contract.set_base_token_burn_percent(context.deployer_account, proxy, 10000)
advance_blocks(1)

GET TOTAL REWARDS FOR WEEK

In [22]:
from utils.utils_chain import WrapperAddress, decode_merged_attributes
from utils import decoding_structures

data_fetcher = FeeCollectorContractDataFetcher(WrapperAddress(fees_collector_contract.address), context.network_provider.proxy.url)
total_rewards = fees_collector_contract.get_total_rewards_for_week(proxy, week)
print(total_rewards)


0


GET ACCUMULATED FEES

In [ ]:
from multiversx_sdk.abi import TokenIdentifierValue, U32Value
token_identifier = "MEX-455c57"
data_fetcher = FeeCollectorContractDataFetcher(WrapperAddress(fees_collector_contract.address), context.network_provider.proxy.url)
accumulated_fees = data_fetcher.get_data("getAccumulatedFees", [U32Value(week), TokenIdentifierValue(token_identifier)])
print(accumulated_fees)


0


GET USER ENERGY FOR WEEK

In [23]:
data_fetcher = FeeCollectorContractDataFetcher(WrapperAddress(fees_collector_contract.address), context.network_provider.proxy.url)
user_energy_for_week = fees_collector_contract.get_user_energy_for_week(user_account.address.to_bech32(), proxy, week)
print(user_energy_for_week)


{'amount': 378166121477748896610405802164, 'last_update_epoch': 1767, 'total_locked_tokens': 410821713011768735240637278}


GET CURRENT CLAIM PROGRESS

In [ ]:
data_fetcher = FeeCollectorContractDataFetcher(WrapperAddress(fees_collector_contract.address), context.network_provider.proxy.url)
current_claim = fees_collector_contract.get_current_claim_progress_for_user(user_account.address.to_bech32(), proxy)
print(current_claim)

{'energy': {'amount': 378166121477748896610405802164, 'last_update_epoch': 1767, 'total_locked_tokens': 410821713011768735240637278}, 'week': 130}


SET ROUTER ADDRESS

In [ ]:
context.deployer_account.sync_nonce(proxy)
fees_collector_contract.set_router_address(context.deployer_account, proxy, router_contract.address)
advance_blocks(5)

In [ ]:
mex_contract.whitelist_contract(context.deployer_account, context.network_provider.proxy, pair_contract.address)

In [ ]:
# set where to swap and what to do with the fees
pair_contract.add_trusted_swap_pair(context.deployer_account, context.network_provider.proxy,
                                    [
                                        mex_contract.address,
                                        mex_contract.firstToken,
                                        mex_contract.secondToken
                                    ])

In [ ]:
from contracts.pair_contract import AddLiquidityEvent

event = AddLiquidityEvent(pair_contract.firstToken, 127791780000000000000, 1, pair_contract.secondToken, 5000000000000000000, 1)
pair_contract.add_liquidity(context.network_provider, user_account, event)

In [ ]:
from utils.utils_chain import base64_to_hex


code_hash = context.network_provider.proxy.get_account(WrapperAddress(fees_collector_contract.address)).contract_code_hash.hex()
print(code_hash)

PAIR SWAP

In [ ]:
from contracts.pair_contract import SwapFixedInputEvent

user_account.sync_nonce(proxy)
swap = SwapFixedInputEvent(pair_contract.firstToken, 1000000000000000000, pair_contract.secondToken, 1)

pair_contract.swap_fixed_input(context.network_provider, user_account, swap)
advance_blocks(1)

In [ ]:
import json
with open(PROJECT_ROOT / "states" / "0_system_account_state_WEGLD-bd4d79.json", "r") as file:
    json_account = json.load(file)
proxy.do_post_generic(f"{SIMULATOR_URL}/simulator/set-state", json_account)

CONTRACT UPGRADE

In [ ]:
from utils.utils_chain import base64_to_hex
from utils.utils_chain import WrapperAddress

def fees_collector_upgrade():
    advance_blocks(1)
    context.deployer_account.address = WrapperAddress("erd1ss6u80ruas2phpmr82r42xnkd6rxy40g9jl69frppl4qez9w2jpsqj8x97")
    context.deployer_account.sync_nonce(proxy)
    args = [ 
        energy_contract.address,
        router_contract.address,
        1000,
        []

    ]
    tx_hash = fees_collector_contract.contract_upgrade(context.deployer_account, context.network_provider.proxy, wasm_path, 
                                         args, 
                                         no_init=True)

    advance_blocks(1)

    code_hash = context.network_provider.proxy.get_account(WrapperAddress(fees_collector_contract.address)).contract_code_hash.hex()
    assert code_hash == contract_code_hash
    
    return tx_hash

fees_collector_upgrade()

SWAP TO BASE TOKEN

In [ ]:
from multiversx_sdk.abi import Abi

abi = Abi.load(PROJECT_ROOT / "wasm/fees-collector/fees-collector.abi.json")

context.deployer_account.sync_nonce(proxy)
    
def build_args_for_swap():

    routes: list[list[Any]] = []
    routes.append((Address(pair_contract.address), "swapTokensFixedInput" , pair_contract.secondToken, 1))
    routes.append((Address(mex_contract.address), "swapTokensFixedInput" , "MEX-455c57", 1))
    
    sc_args = [
           pair_contract.firstToken,
           routes
        ]

    return sc_args

args = build_args_for_swap()

fees_collector_contract.swap_to_base_token(context.deployer_account, proxy, abi, args)
advance_blocks(1)

In [ ]:
advance_blocks(1)

ADD REWARD TOKENS

In [ ]:
context.deployer_account.sync_nonce(proxy)
fees_collector_contract.add_reward_tokens(context.deployer_account, proxy, ["MEX-455c57", "XMEX-fda355"])

REMOVE REWARD TOKENS

In [ ]:
fees_collector_contract.remove_reward_tokens(context.deployer_account, proxy, ["USDC-c76f1f",
                                                                               "RIDE-7d18e9", 
                                                                               "CRU-a5f4aa", 
                                                                               "ZPAY-247875", 
                                                                               "ITHEUM-df6f26", 
                                                                               "BHAT-c1fde3", 
                                                                               "CRT-52decf", 
                                                                               "UTK-2f80e9",
                                                                               "QWT-46ac01",
                                                                               "ASH-a642d1",
                                                                               "WETH-b4ca29",
                                                                               "USDT-f8c08c",
                                                                               "HTM-f51d55",
                                                                               "WDAI-9eeb54",
                                                                               "TADA-5c032c",
                                                                               "XOXNO-c1293a",
                                                                               "A1X-0d446d",
                                                                               "USH-111e09",
                                                                               "FOXSY-5d5f3e"
                                                                               ])
advance_blocks(1)

In [ ]:
energy_factory: SimpleLockEnergyContract
energy_factory = context.get_contracts(config.SIMPLE_LOCKS_ENERGY)[0]

context.deployer_account.sync_nonce(proxy)
tx_hash = energy_factory.set_burn_role_locked_token(context.deployer_account,
                                                                proxy,
                                                                [fees_collector_contract.address])

advance_blocks(1)

In [ ]:
energy_factory.set_transfer_role_locked_token(context.deployer_account, context.network_provider.proxy, [mex_contract.address])
advance_blocks(1)

In [ ]:
from contracts.builtin_contracts import ESDTContract

esdt_contract = ESDTContract(config.TOKENS_CONTRACT_ADDRESS)
context.deployer_account.sync_nonce(context.network_provider.proxy)
tx_hash = esdt_contract.set_special_role_token(context.deployer_account, context.network_provider.proxy,
                                               [BASE_TOKEN, fees_collector_contract.address, "ESDTRoleLocalMint", "ESDTRoleLocalBurn"])
